In [1]:
# !pip install langchain==0.0.346
# !pip install newspaper3k==0.2.8
# !pip install pandas==2.1.3
# !pip install deltalake==0.14.0
# !pip install tabulate==0.9.0
# !pip install googlenews
# !pip install gnews==0.3.6
# !pip install plotly==5.18.0
# ! pip install nbformat==5.9.2
# !pip show nbformat

In [2]:
import sys
sys.path.append('..')

import os
import pandas as pd

from tabulate import tabulate
from datetime import datetime, timedelta
import pytz

from deltalake import DeltaTable
from deltalake.writer import write_deltalake

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
# Params
table_path = '../data/articles-ingest'
# --------------------------------------------
query = "Ozempic Eli Lilly"

days_lookback = 30*2
max_results = 1000
current_datetime = datetime.now().astimezone(pytz.utc)
current_date = current_datetime.date()
start_date = (current_datetime - timedelta(days=days_lookback)).date()

# --------------------------------------------

start_date_str = str(start_date.strftime("%Y-%m-%d"))
current_date_str = str(current_date.strftime("%Y-%m-%d"))

print(
    f"""
    end_date: {current_date}
    start_date: {start_date}
    
    
    days_lookback: {days_lookback}
    max_results: {max_results}
    """
)


    end_date: 2023-12-07
    start_date: 2023-10-08
    
    
    days_lookback: 60
    max_results: 1000
    


In [4]:
# Query queries that have been ingested
# DeltaTable(table_path).to_pandas(columns=['query']).value_counts()

In [5]:
df = DeltaTable(table_path).to_pandas(
    filters=[
        ('query','=',query),
        ('published date str','>', start_date_str),
    ]
)
df

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article text,article html,article summary,article keywords
0,Eli Lilly Just Got One Step Closer to Dethroni...,Eli Lilly Just Got One Step Closer to Dethroni...,2023-11-19,https://news.google.com/rss/articles/CBMiXWh0d...,"{'href': 'https://www.fool.com', 'title': 'The...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-19 08:00:00,2023-11-19,Just about every evening when I sit down for d...,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",Let's break down what Eli Lilly has in its pip...,"[step, eli, closer, drug, lilly, ozempic, buy,..."
1,Ozempic Sales Are Skyrocketing Faster Than Exp...,Ozempic Sales Are Skyrocketing Faster Than Exp...,2023-10-17,https://news.google.com/rss/articles/CBMiXWh0d...,"{'href': 'https://www.fool.com', 'title': 'The...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-10-17 07:00:00,2023-10-17,Everything seems to be going right for Novo No...,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",Ozempic sales are skyrocketing faster than exp...,"[expected, drug, skyrocketing, lilly, ozempic,..."
2,Will Ozempic-type obesity drugs pay off for so...,Will Ozempic-type obesity drugs pay off for so...,2023-10-17,https://news.google.com/rss/articles/CBMiUWh0d...,"{'href': 'https://www.statnews.com', 'title': ...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-10-17 07:00:00,2023-10-17,"Our expensive obesity drugs are worth it, Novo...","<!DOCTYPE html>\n<html lang=""en-US"">\n\t<head>...","Our expensive obesity drugs are worth it, Novo...","[studies, eli, lilly, wide, worth, novo, expen..."
3,"New Playground, New Court, New Classroom? Than...","New Playground, New Court, New Classroom? Than...",2023-10-17,https://news.google.com/rss/articles/CBMiVGh0d...,"{'href': 'https://www.wsj.com', 'title': 'The ...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-10-17 07:00:00,2023-10-17,None,None,None,None
4,Is Eli Lilly Stock A Buy After Mounjaro Tops O...,Is Eli Lilly Stock A Buy After Mounjaro Tops O...,2023-11-28,https://news.google.com/rss/articles/CBMiQmh0d...,"{'href': 'https://www.investors.com', 'title':...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-28 20:14:00,2023-11-28,Eli Lilly (LLY) stock jumped in early November...,"<!doctype html>\n<html class=""no-js"" lang=""en-...",The news lifted Point stock and shares of Lant...,"[diabetes, eli, drug, tops, mounjaro, ozempic,..."
5,Eli Lilly Has Yet Another Ozempic Killer in th...,Eli Lilly Has Yet Another Ozempic Killer in th...,2023-11-09,https://news.google.com/rss/articles/CBMiU2h0d...,"{'href': 'https://www.fool.com', 'title': 'The...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-09 08:00:00,2023-11-09,Eli Lilly (LLY 0.19%) and Novo Nordisk (NVO -2...,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",Eli Lilly (LLY 0.19%) and Novo Nordisk (NVO -2...,"[markets, eli, killer, lilly, works, ozempic, ..."
6,Eli Lilly’s new weight loss drug could drive d...,Eli Lilly’s new weight loss drug could drive d...,2023-11-09,https://news.google.com/rss/articles/CBMiiQFod...,"{'href': 'https://www.dallasnews.com', 'title'...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-09 08:00:00,2023-11-09,An effective and massively popular weight-loss...,"<!DOCTYPE html><html lang=""en""><head><script t...",An effective and massively popular weight-loss...,"[diabetes, eli, drug, price, higher, lilly, oz..."
7,"Ozempic for Weight Loss: No Problem, Restauran...","Ozempic for Weight Loss: No Problem, Restauran...",2023-11-09,https://news.google.com/rss/articles/CBMib2h0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Ozempic Eli Lilly,2023-12-07 02:38:25.798191,2023-11-09 08:00:00,2023-11-09,Bloomberg Daybreak Asia\n\nLive market coverag...,"<!DOCTYPE html><html lang=""en""><head><meta cha...",Bloomberg Daybreak AsiaLive market coverage co...,"[restaurants, markets, diners, ozempic, buy, o..."
8,This Drug Could Be the Nex

In [27]:
df.columns

Index(['title', 'description', 'published date', 'url', 'publisher', 'query',
       'query end date', 'published datetime', 'published date str',
       'article text', 'article html', 'article summary', 'article keywords'],
      dtype='object')

In [28]:

i=0
article_text = df.iloc[i]['article text']
published_date_str = df.iloc[i]['published date str']
title = df.iloc[i]['title']
query = df.iloc[i]['query']

In [38]:
def generate_sentiment_prompt(query, title, article_text, published_date_str):
    prompt = f"""
    You are a reviewer of articles. 
    
    Please read the article below and give me a sentiment score between 0 and 10, where 0 is very negative and 10 is very positive.
    Please output this number, and a short explanation of why you gave this score.

    Search term used to find this article: 
    {query}

    The article:

    Title: {title}
    Published Date: {published_date_str}
    Article: {article_text}

    Please return the output as a JSON object with the following keys: sentiment_score, sentiment_explanation
    """
    return prompt

In [39]:
generate_sentiment_prompt(query, title, article_text, published_date_str)

"\n    You are a reviewer of articles. \n    \n    Please read the article below and give me a sentiment score between 0 and 10, where 0 is very negative and 10 is very positive.\n    Please output this number, and a short explanation of why you gave this score.\n\n    Search term used to find this article: \n    Ozempic Eli Lilly\n\n    The article:\n\n    Title: Eli Lilly Just Got One Step Closer to Dethroning Ozempic. Time to ... - The Motley Fool\n    Published Date: 2023-11-19\n    Article: Just about every evening when I sit down for dinner and turn on the television, a series of commercials with catchy jingles grabs my attention. It's always the same ads, each promoting a treatment from a pharmaceutical company. The most recognizable commercials at this point are for Ozempic and Rybelsus, both of which are weight-loss supplements developed by Danish drug company Novo Nordisk.\n\nHowever, Novo Nordisk isn't the only pharmaceutical company dominating the airwaves. Ozempic's top ri